In [5]:
!pip install arxiv nltk tqdm

In [16]:
import fitz  # PyMuPDF for PDF manipulation
import os
import random
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from tqdm import tqdm

# Download necessary NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading 'punkt' for NLTK...")
    nltk.download('punkt')

# Create directories if they don’t exist
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# List of common academic phrases with subtle errors
ACADEMIC_PHRASE_REPLACEMENTS = {
    "significant results": "somewhat significant results",
    "clearly demonstrates": "possibly demonstrates",
    "strongly correlated": "weakly correlated",
    "highly effective": "moderately effective",
    "novel approach": "somewhat novel approach",
    "state-of-the-art": "near state-of-the-art",
    "outperforms": "slightly outperforms",
    "substantial improvement": "minor improvement",
    "robust performance": "inconsistent performance",
    "conclusive evidence": "preliminary evidence",
    "optimal solution": "suboptimal solution",
    "comprehensive analysis": "partial analysis",
    "rigorous evaluation": "limited evaluation",
    "critical insight": "potential insight",
    "innovative method": "derivative method",
    "experimental validation": "partial validation",
    "theoretical foundation": "theoretical assumption",
    "remarkable efficiency": "reasonable efficiency",
    "promising results": "mixed results",
    "consistent with prior work": "somewhat consistent with prior work",
}

# Slightly alter citation formatting
def degrade_citations(text):
    citation_patterns = [
        (r'\[(\d+)\]', r'[Ref. \1]'),
        (r'\(([A-Za-z]+) et al\., (\d{4})\)', r'(\1 et al \2)'),
        (r'\(([A-Za-z]+) and ([A-Za-z]+), (\d{4})\)', r'(\1 & \2 \3)'),
    ]
    
    degraded = text
    for pattern, replacement in citation_patterns:
        if random.random() < 0.7:
            degraded = re.sub(pattern, replacement, degraded)
    
    return degraded

# Introduce subtle grammar and punctuation issues
def degrade_grammar_punctuation(sentence):
    if random.random() > 0.3:
        return sentence
    
    degradations = [
        lambda s: re.sub(r', ', ' ', s, count=1) if ', ' in s else s,
        lambda s: s.replace(' the ', ' the the ', 1) if ' the ' in s else s,
        lambda s: s.replace('. ', '; ', 1) if '. ' in s else s.replace('; ', '. ', 1) if '; ' in s else s,
        lambda s: s.replace(' is ', ' is essentially ', 1) if ' is ' in s else s,
        lambda s: s.replace(' demonstrates ', ' demonstrated ', 1) if ' demonstrates ' in s else s,
    ]
    
    degradation = random.choice(degradations)
    return degradation(sentence)

# Introduce subtle statistical reporting issues
def degrade_statistics(text):
    p_val_pattern = r'(p\s*<\s*0\.0\d{1,2})'
    r_sq_pattern = r'(R\s*\^\s*2\s*=\s*0\.\d{1,2})'
    
    def alter_p_value(match):
        if random.random() > 0.6:
            return match.group(0)
        p_val = match.group(0)
        return p_val.replace('0.0', '0.0') + '5'
    
    def alter_r_squared(match):
        if random.random() > 0.6:
            return match.group(0)
        r_sq = match.group(0)
        last_digit = int(r_sq[-1])
        if last_digit > 0:
            return r_sq[:-1] + str(last_digit - 1)
        return r_sq
    
    degraded = re.sub(p_val_pattern, alter_p_value, text)
    degraded = re.sub(r_sq_pattern, alter_r_squared, degraded)
    
    return degraded

# Introduce occasional terminology inconsistencies
def degrade_terminology(text):
    common_terms = {
        "neural network": ["neural net", "neural-network", "NN"],
        "deep learning": ["DL", "deep-learning", "deep neural network"],
        "machine learning": ["ML", "machine-learning", "statistical learning"],
        "artificial intelligence": ["AI", "artificial-intelligence", "computational intelligence"],
        "convolutional neural network": ["CNN", "ConvNet", "convolutional network"],
        "recurrent neural network": ["RNN", "recurrent net", "sequential neural network"],
        "dataset": ["data set", "data", "training set"],
        "algorithm": ["method", "approach", "technique"],
        "parameter": ["parameter value", "hyperparameter", "model parameter"],
        "experiment": ["experimental setup", "empirical evaluation", "test"],
    }
    
    degraded = text
    terms_to_replace = random.sample(list(common_terms.keys()), min(3, len(common_terms)))
    
    for term in terms_to_replace:
        if term in degraded and random.random() < 0.6:
            replacement = random.choice(common_terms[term])
            pattern = r'\b' + re.escape(term) + r'\b'
            match = re.search(pattern, degraded)
            if match:
                pos = match.start()
                degraded = degraded[:pos] + replacement + degraded[pos+len(term):]
    
    return degraded

# Slightly modify equations (fixed invalid group reference)
def degrade_equations(text):
    equation_patterns = [
        (r'([A-Za-z])\s*=\s*([A-Za-z0-9]+)\s*\+\s*([A-Za-z0-9]+)', r'\1 = \2 - \3'),  # Change + to -
        (r'([A-Za-z])\s*=\s*([A-Za-z0-9]+)\s*\/\s*([A-Za-z0-9]+)', r'\1 = \2 * \3'),  # Change / to *
        (r'([A-Za-z])\s*\^\s*2', r'\1^3'),  # Change squared to cubed (fixed: no \3 reference)
    ]
    
    degraded = text
    for pattern, replacement in equation_patterns:
        try:
            if random.random() < 0.4:
                degraded = re.sub(pattern, replacement, degraded, count=1)
        except re.error as e:
            tqdm.write(f"Regex error in degrade_equations: {str(e)}. Skipping this pattern.")
    
    return degraded

# Introduce subtle formatting issues in lists and enumerations
def degrade_lists(text):
    numbered_pattern = r'(\d+)\.\s+([A-Z][^\.]+\.)'
    bullet_pattern = r'•\s+([A-Z][^\.]+\.)'
    
    if random.random() < 0.5:
        degraded = re.sub(numbered_pattern, r'• \2', text, count=random.randint(1, 2))
    else:
        start_num = random.randint(2, 5)
        degraded = text
        bullet_matches = list(re.finditer(bullet_pattern, text))
        
        for i, match in enumerate(bullet_matches[:2]):
            if random.random() < 0.7:
                pos = match.start()
                replacement = f"{start_num + i}. {match.group(1)}"
                degraded = degraded[:pos] + replacement + degraded[pos+len(match.group(0)):]
    
    return degraded

# Comprehensive text degradation with subtle academic quality issues
def degrade_text_academic(text):
    if not text or len(text) < 100:
        return text
    
    try:
        sentences = sent_tokenize(text)
        paragraphs = []
        current_paragraph = []
        
        for sent in sentences:
            current_paragraph.append(sent)
            if len(sent) > 50 and sent.endswith('.') and random.random() < 0.3:
                paragraphs.append(' '.join(current_paragraph))
                current_paragraph = []
        
        if current_paragraph:
            paragraphs.append(' '.join(current_paragraph))
        
        degraded_paragraphs = []
        
        for paragraph in paragraphs:
            for phrase, replacement in ACADEMIC_PHRASE_REPLACEMENTS.items():
                if phrase in paragraph and random.random() < 0.4:
                    paragraph = paragraph.replace(phrase, replacement, 1)
            
            paragraph = degrade_citations(paragraph)
            
            sentences = sent_tokenize(paragraph)
            degraded_sentences = []
            
            for sent in sentences:
                sent = degrade_grammar_punctuation(sent)
                degraded_sentences.append(sent)
            
            degraded_paragraph = ' '.join(degraded_sentences)
            degraded_paragraph = degrade_terminology(degraded_paragraph)
            degraded_paragraph = degrade_statistics(degraded_paragraph)
            degraded_paragraph = degrade_equations(degraded_paragraph)
            degraded_paragraph = degrade_lists(degraded_paragraph)
            
            degraded_paragraphs.append(degraded_paragraph)
        
        final_paragraphs = []
        i = 0
        while i < len(degraded_paragraphs):
            if i < len(degraded_paragraphs) - 1 and random.random() < 0.2:
                final_paragraphs.append(degraded_paragraphs[i] + ' ' + degraded_paragraphs[i+1])
                i += 2
            elif len(degraded_paragraphs[i].split()) > 100 and random.random() < 0.3:
                sentences = sent_tokenize(degraded_paragraphs[i])
                split_point = len(sentences) // 2
                final_paragraphs.append(' '.join(sentences[:split_point]))
                final_paragraphs.append(' '.join(sentences[split_point:]))
                i += 1
            else:
                final_paragraphs.append(degraded_paragraphs[i])
                i += 1
        
        for i, para in enumerate(final_paragraphs):
            if "abstract" in para.lower()[:50] or "introduction" in para.lower()[:50]:
                final_paragraphs[i] = para.replace("demonstrates", "suggests").replace("proves", "indicates")
            elif "conclusion" in para.lower()[:50] or "discussion" in para.lower()[:50]:
                final_paragraphs[i] = para.replace("future work", "possible future direction").replace("clearly shows", "appears to show")
        
        for i, para in enumerate(final_paragraphs):
            if "references" in para.lower()[:50] or "bibliography" in para.lower()[:50]:
                refs = para.split("\n")
                degraded_refs = []
                for ref in refs:
                    if random.random() < 0.3:
                        if "et al." in ref:
                            ref = ref.replace("et al.", "et al")
                        elif ", pp." in ref:
                            ref = ref.replace(", pp.", ", p.")
                        elif "(" in ref and ")" in ref:
                            ref = ref.replace("(", "[").replace(")", "]")
                    degraded_refs.append(ref)
                final_paragraphs[i] = "\n".join(degraded_refs)
        
        result = "\n\n".join(final_paragraphs)
        return result
    
    except Exception as e:
        tqdm.write(f"Error during academic degradation: {str(e)}. Returning original text.")
        return text

# Improved PDF text extraction
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        
        for page_num in range(len(doc)):
            try:
                page = doc.load_page(page_num)
                page_text = page.get_text("text")
                
                if page_text:
                    text += page_text + "\n"
                else:
                    raw_dict = page.get_text("rawdict")
                    if raw_dict and "blocks" in raw_dict:
                        for block in raw_dict["blocks"]:
                            if "lines" in block:
                                for line in block["lines"]:
                                    if "spans" in line:
                                        for span in line["spans"]:
                                            if "text" in span:
                                                text += span["text"] + " "
            except Exception as e:
                tqdm.write(f"Error extracting text from page {page_num}: {str(e)}")
        
        doc.close()
        clean_text = text.encode('utf-8', errors='replace').decode('utf-8', errors='replace')
        return clean_text
    except Exception as e:
        tqdm.write(f"Error opening or processing PDF: {str(e)}")
        return ""

# Extract structure information from PDF
def extract_pdf_structure(pdf_path):
    structure = {
        "title": "",
        "abstract": "",
        "sections": [],
        "has_equations": False,
        "has_tables": False,
        "has_figures": False,
        "has_references": False
    }
    
    try:
        doc = fitz.open(pdf_path)
        text = extract_text_from_pdf(pdf_path)
        
        lines = text.split('\n')
        current_section = None
        
        for i, line in enumerate(lines):
            if i < 5 and not structure["title"] and len(line.strip()) > 10 and line.strip().isupper():
                structure["title"] = line.strip()
            
            if "abstract" in line.lower() and i < 20:
                structure["abstract"] = True
            
            if re.match(r'^\d+\.\s+[A-Z]', line) or line.strip().isupper() and len(line.strip()) < 40:
                current_section = line.strip()
                structure["sections"].append(current_section)
            
            if "table" in line.lower() and ":" in line:
                structure["has_tables"] = True
            if "figure" in line.lower() and ":" in line:
                structure["has_figures"] = True
            if "reference" in line.lower() or "bibliography" in line.lower():
                structure["has_references"] = True
            if re.search(r'[a-zA-Z]\s*[=<>]\s*[a-zA-Z0-9]', line) or re.search(r'\([a-zA-Z0-9]+\)', line):
                structure["has_equations"] = True
        
        common_sections = ["introduction", "related work", "method", "experiment", "result", "discussion", "conclusion"]
        structure["has_standard_structure"] = any(section.lower() in ' '.join(structure["sections"]).lower() for section in common_sections)
        
        doc.close()
        return structure
    except Exception as e:
        tqdm.write(f"Error extracting structure: {str(e)}")
        return structure

# Create a degraded PDF with subtle academic quality issues
def create_degraded_pdf(original_text, structure, output_path):
    try:
        degraded_doc = fitz.open()
        y_pos = 50
        margin = 72
        
        page = degraded_doc.new_page()
        if structure["title"]:
            page.insert_text((margin, y_pos), structure["title"], fontsize=14, fontname="Times-Roman")
            y_pos += 30
        
        paragraphs = original_text.split('\n\n')
        current_page = page
        
        for paragraph in paragraphs:
            if not paragraph.strip():
                continue
                
            if len(paragraph) < 50 and (paragraph.isupper() or re.match(r'^\d+\.\s+', paragraph)):
                fontsize = 12
                if random.random() < 0.3:
                    fontsize = 11 if random.random() < 0.5 else 13
                
                y_pos += random.randint(10, 20)
                
                if y_pos > 750:
                    current_page = degraded_doc.new_page()
                    y_pos = 50
                
                current_page.insert_text((margin, y_pos), paragraph, fontsize=fontsize, fontname="Times-Bold")
                y_pos += 20
                continue
            
            fontsize = 10
            if random.random() < 0.2:
                fontsize = 9.5 if random.random() < 0.5 else 10.5
            
            words = paragraph.split()
            current_line = []
            line_width = 0
            max_width = 500
            
            for word in words:
                word_width = len(word) * fontsize * 0.5
                if line_width + word_width > max_width:
                    if y_pos > 750:
                        current_page = degraded_doc.new_page()
                        y_pos = 50
                    
                    current_page.insert_text((margin, y_pos), ' '.join(current_line), fontsize=fontsize, fontname="Times-Roman")
                    y_pos += int(fontsize * 1.2)
                    current_line = [word]
                    line_width = word_width
                else:
                    current_line.append(word)
                    line_width += word_width + 5
            
            if current_line:
                if y_pos > 750:
                    current_page = degraded_doc.new_page()
                    y_pos = 50
                
                current_page.insert_text((margin, y_pos), ' '.join(current_line), fontsize=fontsize, fontname="Times-Roman")
                y_pos += int(fontsize * 1.5)
        
        degraded_doc.save(output_path)
        degraded_doc.close()
        return True
    except Exception as e:
        tqdm.write(f"Error creating degraded PDF: {str(e)}")
        return False

# Process existing PDFs and degrade them
def create_degraded_dataset(original_dir="original_pdfs", degraded_dir="degraded_pdfs"):
    ensure_dir(degraded_dir)
    
    print(f"Processing PDFs from {original_dir}...")
    pdf_files = [f for f in os.listdir(original_dir) if f.endswith('.pdf')]
    total_papers = len(pdf_files)
    print(f"Found {total_papers} papers to process.")
    
    if total_papers == 0:
        print(f"No PDF files found in {original_dir}. Please add PDFs to the directory.")
        return
    
    successful = 0
    for i, pdf_file in enumerate(tqdm(pdf_files, desc="Processing Papers", unit="paper")):
        pdf_path = os.path.join(original_dir, pdf_file)
        tqdm.write(f"Processing: {pdf_path}")
        
        try:
            original_text = extract_text_from_pdf(pdf_path)
            
            if not original_text or len(original_text) < 500:
                tqdm.write(f"Insufficient text extracted from {pdf_file}. Skipping.")
                continue
            
            structure = extract_pdf_structure(pdf_path)
            degraded_text = degrade_text_academic(original_text)
            
            degraded_filename = f"degraded_{pdf_file}"
            degraded_path = os.path.join(degraded_dir, degraded_filename)
            if create_degraded_pdf(degraded_text, structure, degraded_path):
                tqdm.write(f"Saved degraded: {degraded_path}")
                successful += 1
            else:
                tqdm.write(f"Failed to create degraded PDF for {pdf_file}.")
        except Exception as e:
            tqdm.write(f"Error processing {pdf_file}: {str(e)}")
    
    print(f"Successfully processed {successful}/{total_papers} papers.")

if __name__ == "__main__":
    create_degraded_dataset(original_dir="original_pdfs", degraded_dir="degraded_pdfs_2")

Processing PDFs from original_pdfs...
Found 395 papers to process.


Processing Papers:   0%|                                                                                                                                   | 0/395 [00:00<?, ?paper/s]

Processing: original_pdfs/paper_313.pdf


Processing Papers:   0%|▎                                                                                                                          | 1/395 [00:00<02:19,  2.82paper/s]

Saved degraded: degraded_pdfs_2/degraded_paper_313.pdf
Processing: original_pdfs/paper_307.pdf


...

Saved degraded: degraded_pdfs_2/degraded_paper_108.pdf
Processing: original_pdfs/paper_322.pdf


Processing Papers: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 394/395 [02:39<00:00,  2.34paper/s]

Saved degraded: degraded_pdfs_2/degraded_paper_322.pdf
Processing: original_pdfs/paper_336.pdf


Processing Papers: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 395/395 [02:39<00:00,  2.48paper/s]

Saved degraded: degraded_pdfs_2/degraded_paper_336.pdf
Successfully processed 395/395 papers.
